<a href="https://colab.research.google.com/github/YichengZou626/COMP590_intro_to_deep_learning/blob/main/Homework_7_(Yicheng_Zou).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 6

In this homework you will be training and using a "char-RNN". This is the name given to a character-level recurrent neural network language model by [this famous blog post by Andrej Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Before you start on the rest of the homework, please give the blog post a read, it's quite good!

I don't expect you to implement the char-RNN from scratch. Andrej's original char-rnn is in Torch (the predecessor to PyTorch that is not commonly used anymore). Fortunately, there are many other implementations of this model available; for example, there is one (in both mxnet and pytorch) in chapters 8 and 9 of [the textbook](http://d2l.ai), and another pytorch one [here](https://github.com/spro/char-rnn.pytorch). **Please use one of these example implementations (or another one that you find) when completing this homework**.

For this homework, please complete the following steps:

1. Download and tokenize the [Shakespeare dataset](http://www.gutenberg.org/files/100/100-0.txt) at a character level. I recommend basing your solution on the following code:
```Python
# Remove non-alphabetical characters, lowercase, and replace whitespace with ' '
raw_dataset = ' '.join(re.sub('[^A-Za-z]+','', text).lower().split())
# Maps token index to character
idx_to_char = list(set(raw_dataset))
# Maps character to token index
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
# Tokenize the dataset
corpus_indices = [char_to_idx[char] for char in raw_dataset]
```
1. Train a "vanilla" RNN (as described in chapter 8 of [the textbook](http://d2l.ai)) on the Shakespeare dataset. Report the training loss and generate some samples from the model at the end of training.
1. Train a GRU RNN (as described in chapter 9 of [the textbook](http://d2l.ai)) on the Shakespeare datatset. Is the final training loss higher or lower than the vanilla RNN? Are the samples from the model more or less realistic?
1. Find a smaller, simpler dataset than the Shakespeare data (you can find some ideas in Andrej's blog post, but feel free to get creative!) and train either the vanilla or GRU RNN on it instead. Is the final training loss higher or lower than it was for the Shakespeare data?

In [ ]:
import re
with open('shake.txt') as f:
  text = f.read()

# Remove non-alphabetical characters, lowercase, and replace whitespace with ' '
raw_dataset = ' '.join(re.sub('[^A-Za-z ]+','', text).lower().split())
# Maps token index to character
idx_to_char = list(set(raw_dataset))
# Maps character to token index
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
# Tokenize the dataset
corpus_indices = [char_to_idx[char] for char in raw_dataset]

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from tqdm import tqdm

import string
import random
import time
import math
import torch

# Turning a string into a tensor

def char_tensor(string):
    c_list = [char_to_idx[char] for char in string]
    tensor = torch.zeros(len(string)).long()
    for c in range(len(c_list)):
        tensor[c] = c_list[c]
    return tensor

# Readable time elapsed

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Use Vanilla RNN

In [ ]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(CharRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        batch_size = input.size(0)
        encoded = self.encoder(input)
        output, hidden = self.rnn(encoded.view(1, batch_size, -1), hidden)
        output = self.decoder(output.view(batch_size, -1))
        return output, hidden

    def forward2(self, input, hidden):
        encoded = self.encoder(input.view(1, -1))
        output, hidden = self.rnn(encoded.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self, batch_size):
        return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))

In [ ]:
def generate(decoder, prime_str='A', predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden(1)
    prime_input = Variable(char_tensor(prime_str).unsqueeze(0))

    hidden = hidden.to(device)
    prime_input = prime_input.to(device)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[:,p], hidden)
        
    inp = prime_input[:,-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]

        # Add predicted character to string and use as next input
        predicted_char = idx_to_char[top_i]
        predicted += predicted_char
        inp = Variable(char_tensor(predicted_char).unsqueeze(0))
        inp = inp.to(device)

    return predicted

In [ ]:
filename = "shake.txt"
n_epochs=2000
print_every=100
hidden_size=100
n_layers=2
learning_rate=0.01
chunk_len=200
batch_size=100


file = raw_dataset
file_len = len(file)

def random_training_set(chunk_len, batch_size):
    inp = torch.LongTensor(batch_size, chunk_len)
    target = torch.LongTensor(batch_size, chunk_len)
    for bi in range(batch_size):
        start_index = random.randint(0, file_len - chunk_len)
        end_index = start_index + chunk_len + 1
        chunk = file[start_index:end_index]
        inp[bi] = char_tensor(chunk[:-1])
        target[bi] = char_tensor(chunk[1:])
    inp = Variable(inp)
    target = Variable(target)
    inp = inp.to(device)
    target = target.to(device)
    return inp, target

def train(inp, target):
    hidden = decoder.init_hidden(batch_size)
    hidden = hidden.to(device)
    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = decoder(inp[:,c], hidden)
        loss += criterion(output.view(batch_size, -1), target[:,c])

    loss.backward()
    decoder_optimizer.step()

    return loss.data / chunk_len

def save():
    save_filename = 'shake1.pt'
    torch.save(decoder, save_filename)
    print('Saved as %s' % save_filename)

# Initialize models and start training

decoder = CharRNN(
    27,
    hidden_size,
    27,
    n_layers,
)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()


decoder.to(device)

start = time.time()
all_losses = []
loss_avg = 0

try:
    print("Training for %d epochs..." % n_epochs)
    for epoch in tqdm(range(1, n_epochs + 1)):
        loss = train(*random_training_set(chunk_len, batch_size))

        if epoch % print_every == 0:
            print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
            print(generate(decoder, 'where', 100, 0.8), '\n')

    print("Saving...")
    save()

except KeyboardInterrupt:
    print("Saving before quit...")
    save()

Training for 2000 epochs...


  5%|▌         | 100/2000 [00:53<16:44,  1.89it/s]

 53s (100 5%) 1.7837]
where wither my prothers of the strocest bose the romed thim in firtuiing and in the good youch aft the l 



 10%|█         | 200/2000 [01:46<16:21,  1.83it/s]

 46s (200 10%) 1.6778]
wheres to the worst so to herelyford great be anmy ful but then and fants deseary and he thy brants sir h 



...

 42s (1800 90%) 1.5521]
where and where fair him stay shame rest right this prey beare there poor exeunt of perceive your seporac 



 95%|█████████▌| 1900/2000 [16:36<00:51,  1.93it/s]

 36s (1900 95%) 1.5482]
where i have all sing the her honour of love ill my still bold for her stay to the leave thee with the he 



100%|██████████| 2000/2000 [17:29<00:00,  1.91it/s]

 29s (2000 100%) 1.5546]
where it say wrong poor under to marles but houset her good speed that reports be the grief thou abolut w 

Saving...
Saved as shake1.pt


In [ ]:
decoder = torch.load('shake1.pt')
generate(decoder, 'where', 100, 0.8)

'where what i may how the sight what let here drimmy could for the case i don exit and to the light the he'

The training loss after 2000 epochs is 1.54.

# Use GRU RNN

In [ ]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, model="gru", n_layers=1):
        super(CharRNN, self).__init__()
        self.model = model.lower()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(input_size, hidden_size)
        if self.model == "gru":
            self.rnn = nn.GRU(hidden_size, hidden_size, n_layers)
        elif self.model == "lstm":
            self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        batch_size = input.size(0)
        encoded = self.encoder(input)
        output, hidden = self.rnn(encoded.view(1, batch_size, -1), hidden)
        output = self.decoder(output.view(batch_size, -1))
        return output, hidden

    def forward2(self, input, hidden):
        encoded = self.encoder(input.view(1, -1))
        output, hidden = self.rnn(encoded.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self, batch_size):
        if self.model == "lstm":
            return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)),
                    Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)))
        return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))

In [ ]:
def generate(decoder, prime_str='A', predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden(1)
    prime_input = Variable(char_tensor(prime_str).unsqueeze(0))

    hidden = hidden.to(device)
    prime_input = prime_input.to(device)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[:,p], hidden)
        
    inp = prime_input[:,-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]

        # Add predicted character to string and use as next input
        predicted_char = idx_to_char[top_i]
        predicted += predicted_char
        inp = Variable(char_tensor(predicted_char).unsqueeze(0))
        inp = inp.to(device)

    return predicted

In [ ]:
filename = "shake.txt"
model="gru"
n_epochs=2000
print_every=100
hidden_size=100
n_layers=2
learning_rate=0.01
chunk_len=200
batch_size=100


file = raw_dataset
file_len = len(file)

def random_training_set(chunk_len, batch_size):
    inp = torch.LongTensor(batch_size, chunk_len)
    target = torch.LongTensor(batch_size, chunk_len)
    for bi in range(batch_size):
        start_index = random.randint(0, file_len - chunk_len)
        end_index = start_index + chunk_len + 1
        chunk = file[start_index:end_index]
        inp[bi] = char_tensor(chunk[:-1])
        target[bi] = char_tensor(chunk[1:])
    inp = Variable(inp)
    target = Variable(target)
    inp = inp.to(device)
    target = target.to(device)
    return inp, target

def train(inp, target):
    hidden = decoder.init_hidden(batch_size)
    hidden = hidden.to(device)
    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = decoder(inp[:,c], hidden)
        loss += criterion(output.view(batch_size, -1), target[:,c])

    loss.backward()
    decoder_optimizer.step()

    return loss.data / chunk_len

def save():
    save_filename = 'shake2.pt'
    torch.save(decoder, save_filename)
    print('Saved as %s' % save_filename)

# Initialize models and start training

decoder = CharRNN(
    27,
    hidden_size,
    27,
    model,
    n_layers,
)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()


decoder.to(device)

start = time.time()
all_losses = []
loss_avg = 0

try:
    print("Training for %d epochs..." % n_epochs)
    for epoch in tqdm(range(1, n_epochs + 1)):
        loss = train(*random_training_set(chunk_len, batch_size))

        if epoch % print_every == 0:
            print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
            print(generate(decoder, 'where', 100, 0.8), '\n')

    print("Saving...")
    save()

except KeyboardInterrupt:
    print("Saving before quit...")
    save()

Training for 2000 epochs...


  5%|▌         | 100/2000 [01:33<30:57,  1.02it/s]

 33s (100 5%) 1.7210]
where is what chick not why glowing nother to to to me good both to th notsce from was forture scrick tru 



 10%|█         | 200/2000 [03:07<28:14,  1.06it/s]

 7s (200 10%) 1.5915]
where melest down and men shall be shoil of the soul and the by a mannes to the like of father to dead wh 



...

 17s (1800 90%) 1.4574]
where being ere the change him and much thou shalt no westmorle it and things to make him i made the heav 



 95%|█████████▌| 1900/2000 [28:44<01:24,  1.19it/s]

 44s (1900 95%) 1.4961]
where no sir of the action what hoit shall be me where is can pursue me and when i be roman thou from my  



100%|██████████| 2000/2000 [30:07<00:00,  1.11it/s]

 7s (2000 100%) 1.4760]
where a selfof the falls from his hand and the duke pedro scived paris pardon that the bosom hearize but  

Saving...
Saved as shake2.pt


In [ ]:
decoder = torch.load('shake2.pt')
generate(decoder, 'where', 100, 0.8)

'where i am a man i am sworn up and soldiers for the better fallswe him in her virtues to london that hath'

The training loss after 2000 epochs is 1.45, which is less than the vanilla RNN model. In that case, use GRU can help the model to train faster.

# Use small dataset of Paul Graham Essays 

In [ ]:
with open('pg_essay.txt') as f:
  text = f.read()

# Remove non-alphabetical characters, lowercase, and replace whitespace with ' '
raw_dataset = ' '.join(re.sub('[^A-Za-z ]+','', text).lower().split())
# Maps token index to character
idx_to_char = list(set(raw_dataset))
# Maps character to token index
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
# Tokenize the dataset
corpus_indices = [char_to_idx[char] for char in raw_dataset]

In [ ]:
filename = "pg_essay.txt"
model="gru"
n_epochs=2000
print_every=100
hidden_size=100
n_layers=2
learning_rate=0.01
chunk_len=200
batch_size=100


file = raw_dataset
file_len = len(file)

def random_training_set(chunk_len, batch_size):
    inp = torch.LongTensor(batch_size, chunk_len)
    target = torch.LongTensor(batch_size, chunk_len)
    for bi in range(batch_size):
        start_index = random.randint(0, file_len - chunk_len)
        end_index = start_index + chunk_len + 1
        chunk = file[start_index:end_index]
        inp[bi] = char_tensor(chunk[:-1])
        target[bi] = char_tensor(chunk[1:])
    inp = Variable(inp)
    target = Variable(target)
    inp = inp.to(device)
    target = target.to(device)
    return inp, target

def train(inp, target):
    hidden = decoder.init_hidden(batch_size)
    hidden = hidden.to(device)
    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = decoder(inp[:,c], hidden)
        loss += criterion(output.view(batch_size, -1), target[:,c])

    loss.backward()
    decoder_optimizer.step()

    return loss.data / chunk_len

def save():
    save_filename = 'pg.pt'
    torch.save(decoder, save_filename)
    print('Saved as %s' % save_filename)

# Initialize models and start training

decoder = CharRNN(
    27,
    hidden_size,
    27,
    model,
    n_layers,
)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()


decoder.to(device)

start = time.time()
all_losses = []
loss_avg = 0

try:
    print("Training for %d epochs..." % n_epochs)
    for epoch in tqdm(range(1, n_epochs + 1)):
        loss = train(*random_training_set(chunk_len, batch_size))

        if epoch % print_every == 0:
            print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
            print(generate(decoder, 'where', 100, 0.8), '\n')

    print("Saving...")
    save()

except KeyboardInterrupt:
    print("Saving before quit...")
    save()

Training for 2000 epochs...


  5%|▌         | 100/2000 [01:31<29:42,  1.07it/s]

 31s (100 5%) 1.5827]
where the seincy looks and their ungeith or one a natence in existing i evice to rights programs in veste 



 10%|█         | 200/2000 [03:03<27:38,  1.09it/s]

 3s (200 10%) 1.4205]
where meeting on the later is this it was they dont iscoop college the day like fand bgsspect but in many 



...

 59s (1800 90%) 1.2434]
where they sad yet someone market befores now i does the worldial things and like quotes who has to live  



 95%|█████████▌| 1900/2000 [27:24<01:33,  1.07it/s]

 24s (1900 95%) 1.2769]
where are always friends with by whatever better in something to say you really best velous of essay thin 



100%|██████████| 2000/2000 [28:48<00:00,  1.16it/s]

 48s (2000 100%) 1.2993]
where now as forexample for matnow of the outcome they came and a company we find on the presocriate to p 

Saving...
Saved as pg.pt


In [ ]:
decoder = torch.load('pg.pt')
generate(decoder, 'where', 100, 0.8)

'where a countried to definitely of the many better and when i dont experiment of the otherif a lot profit'

The training loss after 2000 epochs is 1.24. Since this time I also use the 

1.   List item
2.   List item

same GRU model, the reason for less training loss is becasue the new text dataset is much smaller than shakespare text, which causes the model train faster. 

# Acknowledgement
The code is adpated from: https://github.com/spro/char-rnn.pytorch

The Paul Graham dataset is downloaded from: https://www.kaggle.com/krsoninikhil/pual-graham-essays